# Infer Maximum Likelihood Estimates of KR model from *in vitro* dataset

In [1]:
import os

import chi
import chi.plots
import numpy as np
import pandas as pd
import pints
import xarray as xr

## Show data

In [2]:
# Import data
directory = os.path.dirname(os.path.dirname(os.getcwd()))
data = pd.read_csv(directory + '/data/who_m_ciprofloxacin.csv')

# Create figure
fig = chi.plots.PDTimeSeriesPlot()
fig.add_data(data)
fig.show()

## Build model

1. Defines the mechanistic model from the SBML file.
2. Defines the error model.
3. Pools parameters across bacterial populations.
4. Fixes unnecessery parameters.

In [3]:
# Define mechanistic model
directory = os.path.dirname(os.path.dirname(os.getcwd()))
mechanistic_model = chi.PharmacokineticModel(
    directory + '/models/KR_model.xml')
mechanistic_model.set_administration(compartment='central')
mechanistic_model.set_parameter_names(names={
    'myokit.bacterial_count_susceptible': 'Initial bacterial count in CFU/ml',
    'myokit.concentration_e50_adapted': 'Adapted EC 50 in ng/ml',
    'myokit.growth_rate': 'Growth rate in 1/h',
    'myokit.kappa_adapted': 'Adapted max. kill rate in 1/h',
    'myokit.kappa_susceptible': 'Wild type kill rate in ml/ng/h',
    'myokit.mutation_rate': 'Mutation rate in ml/ng/h'})
mechanistic_model.set_outputs(['myokit.total_bacterial_count'])
mechanistic_model.set_output_names({
    'myokit.total_bacterial_count': 'Bacterial count in CFU/ml'})

# Define error model
error_model = chi.LogNormalErrorModel()  # Bacterial count

# Define population model
population_models = [
    chi.PooledModel(),  # Initial bacterial count in CFU/ml
    chi.PooledModel(),  # Adapted EC 50 in ng/ml
    chi.PooledModel(),  # Growth rate in 1/h
    chi.PooledModel(),  # Adapted max. kill rate in 1/h
    chi.PooledModel(),  # Wild type kill rate in ml/ng/h
    chi.PooledModel(),  # Mutation rate in ml/ng/h
    chi.PooledModel()]  # Sigma rel. Bacterial count

# Compose model and fix not identified parameters
problem = chi.ProblemModellingController(mechanistic_model, error_model)
problem.fix_parameters({
    'central.drug_amount': 0,
    'myokit.bacterial_count_adapted': 0,
    'central.size': 1,
    'myokit.elimination_rate': 0,
    'myokit.gamma': 1})
problem.set_population_model(population_models)

## Prior predictive check

1. Bounds parameters to realistic orders of magnitudes.
2. Checks that parameters within these bounds can lead to feasible bacterial
    count predictions.

In [4]:
# Define prior distribution
log_priors = [
    pints.UniformLogPrior(0, 1E6),      # Initial bacterial count in CFU/ml
    pints.UniformLogPrior(0, 1000),     # Adapted EC 50 in ng/ml
    pints.UniformLogPrior(0, 5),        # Growth rate in 1/h
    pints.UniformLogPrior(0, 5),        # Adapted max. kill rate in 1/h
    pints.UniformLogPrior(0, 1E-2),     # Wild type kill rate in ml/ng/h
    pints.UniformLogPrior(0, 1E-4),     # Mutation rate in ml/ng/h
    pints.UniformLogPrior(0, 3)]        # Sigma log Bacterial count
log_prior = pints.ComposedLogPrior(*log_priors)

# Compose prior predictive model
predictive_model = problem.get_predictive_model(exclude_pop_model=True)
prior_predictive_model = chi.PriorPredictiveModel(
    predictive_model, log_prior)

# Set dosing regimen (dose 10 mg is similar to ID 10)
prior_predictive_model.set_dosing_regimen(dose=10000, start=4, period=6, num=5)

# Sample from prior predictive model
n_samples = 100
times = np.linspace(0, 10, num=200)
samples = prior_predictive_model.sample(times, n_samples)

# Illustrate prior predictive model
fig = chi.plots.PDPredictivePlot()
fig.add_prediction(
    data=samples,
    biomarker='Bacterial count in CFU/ml',
    bulk_probs=[0.3, 0.6, 0.9])
fig.set_axis_labels(
    xlabel=r'$\text{Time in hours}$',
    ylabel=r'$\text{Bacterial count in CFU/ml}$')
fig.show()

## Find maximum likelihood estimates

1. Connects the above defined model to the data.
2. Runs the maximum likelihood optimisation 5 times from random initial
    starting points (for robustness). Initial points are samples from priors.

Note that because we chose uniform priors, the maximum likelihood estimates
(for the specified bounds) coincide with the maximum a posteriori estimates.

In [5]:
# Create log-posteriors
problem.set_data(data, dose_key='Concentration', dose_duration_key=None)
problem.set_log_prior(log_priors)
log_posterior = problem.get_log_posterior()

# Set up sampling controller
optimiser = chi.OptimisationController(log_posterior)
optimiser.set_optimiser(pints.CMAES)
optimiser.set_transform(
    transform=pints.LogTransformation(n_parameters=problem.get_n_parameters()))
optimiser.set_n_runs(5)

# Run optimisation
estimates = optimiser.run(show_run_progress_bar=False, log_to_screen=False)

# Create figure
fig = chi.plots.ParameterEstimatePlot()
fig.add_data(estimates)
fig.show()

The estimates across the runs are consistent within 3 significant digits. We
can therefore assume that MLEs within the bounds are robust.

## Goodness of fit
1. Predict bacterial count for observed drug exposure.
2. Visualise residuals between model prediction and data.

In [6]:
# Fix unnecessary parameters of mechanistic model
mechanistic_model = chi.ReducedMechanisticModel(
    mechanistic_model)
mechanistic_model.fix_parameters({
    'central.drug_amount': 0,
    'myokit.bacterial_count_adapted': 0,
    'central.size': 1,
    'myokit.elimination_rate': 0,
    'myokit.gamma': 1})

# Get MLE parameters (and fill in fixed parameters)
mle_score = estimates['Score'].max()
mask = estimates['Score'] == mle_score
mle_estimates = estimates[mask]
parameters = [
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Initial bacterial count in CFU/ml',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Adapted EC 50 in ng/ml',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Growth rate in 1/h',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Adapted max. kill rate in 1/h',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Wild type kill rate in ml/ng/h',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Mutation rate in ml/ng/h',
        'Estimate'].iloc[0]
]

# Get dosing regigmes
regimens = problem.get_dosing_regimens()

# Get relevant measurements
mask = data['Biomarker'] == 'Bacterial count'
temp = data[mask]

# Construct predictions for all dosing regimens
predictions = pd.DataFrame(
    columns=['ID', 'Time', 'Biomarker', 'Sample'])
ids = temp['ID'].dropna().unique()
for _id in ids:
    # Set regimen
    regimen = regimens[str(_id)]
    mechanistic_model.simulator.set_protocol(regimen)

    # Get relevant times
    mask = temp['ID'] == _id
    times = temp.loc[mask, 'Time'].to_numpy()
    pred = mechanistic_model.simulate(
        parameters=parameters, times=times)
    
    # Store predictions in dataframe
    predictions = predictions.append(pd.DataFrame({
        'ID': _id,
        'Time': times,
        'Biomarker': 'Bacterial count',
        'Sample': pred.flatten()
    }))

# Create figure
fig = chi.plots.PDPredictivePlot()
fig.add_data(predictions, meas_key='Sample')
fig.show()

Visualise residuals to measurements

In [7]:
# Create figure
fig = chi.plots.ResidualPlot(measurements=temp)
fig.add_data(data=predictions, show_relative=False)
fig._fig.update_layout(xaxis_type='log')
fig.show()

## Estimate cross entropy using AIC

In [9]:
# Sort parameters
parameters = [
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Initial bacterial count in CFU/ml',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Adapted EC 50 in ng/ml',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Growth rate in 1/h',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Adapted max. kill rate in 1/h',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Wild type kill rate in ml/ng/h',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Mutation rate in ml/ng/h',
        'Estimate'].iloc[0],
    mle_estimates.loc[
        mle_estimates['Parameter'] == 'Pooled Sigma log',
        'Estimate'].iloc[0]
]

# Compute AIC score
log_likelihood = log_posterior.get_log_likelihood()
n_parameters = log_likelihood.n_parameters()
n_measurements = np.sum(log_likelihood.n_observations())
AIC = \
    -2 * log_likelihood(parameters) \
    + 2 * n_measurements * n_parameters / (n_measurements - n_parameters - 1)
AIC

2640.0303959758057

### Save inference data

In [10]:
# Format estimates
inf_data = xr.DataArray(
    data=parameters,
    dims=['estimate'],
    coords=dict(
        estimate=[
            'Pooled Initial bacterial count in CFU/ml',
            'Pooled Adapted EC 50 in ng/ml',
            'Pooled Growth rate in 1/h',
            'Pooled Adapted max. kill rate in 1/h',
            'Pooled Wild type kill rate in ml/ng/h',
            'Pooled Mutation rate in ml/ng/h',
            'Pooled Sigma log'])
)
inf_data.attrs['AIC score'] = AIC

# Save estimates
directory = os.getcwd()
path = os.path.join(
    directory, 'derived_data/KR_model_inference_data.nc')
inf_data.to_netcdf(path)